In [ ]:
import tensorflow as tf # tensorflow v2.11.0
import numpy as np

In [ ]:
import re
from sklearn.preprocessing import OneHotEncoder
from typing import NamedTuple

EmbeddingShape = NamedTuple("EmbeddingShape", [("vocab_size", int), ("embedding_size", int)])
'''(NamedTuple) class for embedding shape contain vocab_size and embedding_size'''

class LogPreprocessor(object):
    """LogPreprocessor
    class for preprocessing data before feed to model
    
    Args:
        record_unknow (bool): whether recording unknown word from preprocessing process or not. Defaults to False.
    """
    
    def __init__(self, record_unknow=False) -> None:
        if record_unknow: 
            self.unknow_words = dict()
            '''dictionary for counting ocurred unknow word from preprocessing process where keys is word and value is occuring number'''
    
    @staticmethod
    def text_cleansing(text):
        """(static method) text_cleansing.
        method for cleansing log text.
        
        Args:
            text (str): log text to cleansing special character.

        Returns:
            str: cleansed log text.
        """
        regex_except_token = r'\B(?!<\w+>\B)[^\w\s]'
        regex_expect_words = r'[^\w<>]+'
        output = re.sub(regex_except_token, '', text)
        output = re.sub(regex_expect_words, ' ', output)
        return output
    
    def load_word2vec_format(self, 
                             file_path, 
                             unknow_token=None, 
                             unknow_repr=None):
        """(instance method) load_word2vec_format
        loading word2vec format file to extract embeadding metrix

        Args:
            file_path (str): file's path to word2vec format file (.txt)
            unknow_token (str, optional): word for represent unknown word (e.g. '<OOV>'). 
            Defaults to None.
            unknow_repr (_ArrayLike, optional): word vector for represent unknown word. 
            if 'unknow_repr' is not set but 'unknow_token' is set 
            it will use zero vector as unknow represent vector. Defaults to None.
        """
        with open(file_path, "r") as f:
            vec = dict()
            for l in f.readlines():
                data = list(filter(None, re.split(" +", l)))
                vec[data[0]] = np.array(data[1:], dtype=np.float32)
            
            f.close()
                
        embedding_size = int(vec.pop(list(vec.keys())[0])[0])
        if unknow_token is not None:
            try:
                unknow_vec = vec[unknow_token]
            except AttributeError:
                print(f"there's not '{unknow_token}' in dictionary.")
                if unknow_repr:
                    assert len(unknow_repr) == embedding_size, \
                        f"unknown represent vector must same shape with embedding size (expected {embedding_size}, got {len(unknow_repr)})"
                else:
                    unknow_repr = [ 0 for _ in range(embedding_size) ]
                vec[unknow_token] = unknow_repr
            self.unknow_token = unknow_token
        else: 
            self.unknow_token = None
        
        vocab_size = len(vec.keys())    
        self.word_vectors = np.array(list(vec.values()))
        self.words_indices = { word: i for i, word in enumerate(vec.keys()) }
        self.embedding_shape = EmbeddingShape(vocab_size, embedding_size)
    
    def indice_encode(self, line):
        """(instance method) indice_encode.
        encoding string words in log line to index number.

        Args:
            line (_ArrayLike[str]): array of tokenized string line.

        Returns:
            List[int]: list contains index number.
        """
        record_unknow = hasattr(self, "unknow_words")
        encoded = list()
        
        
        for word in line:
            try:
                i = self.words_indices[word]
            except KeyError:
                if record_unknow:
                    if word in self.unknow_words.keys(): self.unknow_words[word] += 1
                    else: self.unknow_words[word] = 0
                
                if self.unknow_token is None: continue
                else: i = self.words_indices[self.unknow_token]
            encoded.append(i)

            
        return encoded
    
    def insert_new_word(self, word, vector, index=-1):
        """(instance method) insert_new_word
        insert a new word and vector to word dictionary at specified index
        
        Args:
            word (str): new word to insert
            vector (_ArrayLike[Float]): new vector to insert
            index (int, optional): insert at index if negative index will insert at (vocab_size + index + 1). Defaults to -1.
        """
        if index < 0: index = self.embedding_shape.vocab_size + index + 1
        assert word not in self.words_indices.keys(), \
            f"there is already exist input word in vocab at {self.words_indices[word]} index"
        assert np.ndim(vector) == 1, \
            f"expect 1 dim vector as a input but got {np.ndim(vector)}."
        assert len(vector) == self.embedding_shape.embedding_size, \
            f"insert vector's shape must match to embedding size. (got {np.shape(vector)})"
            
        new_word_vectors = np.insert(self.word_vectors, index, vector, axis=0)
        new_words_indices = { k: v if v < index else v + 1 for k, v in self.words_indices.items() }
        new_words_indices[word] = index
        new_embedding_shape = EmbeddingShape(self.embedding_shape.vocab_size + 1, self.embedding_shape.embedding_size)
        
        self.word_vectors = new_word_vectors
        self.words_indices = new_words_indices
        self.embedding_shape = new_embedding_shape
    
    def indice_padding(self, batch, padding_token, padding_size=0):
        """(instance method) indice_padding.
        padding tokenized array in batch by index number of padding_token.
        
        Args:
            batch (_ArrayLike[_ArrayLike[int]]): data batch of array of index number.
            padding_token (str): word for represent padding token (e.g. "<PADDING>") where padding token must in dictionary.
            padding_size (int, optional): size of output. if 'padding_size' is less than longest line in batch it will set to length of longest line is batch. Defaults to 0.

        Returns:
            NDArray[NDArray[int]]: array of padded batch data.
        """
        padding_idx = self.words_indices[padding_token]
        seq_length = np.array([ len(inst) for inst in batch ])
        max_length = seq_length.max()
        padding_size = max(padding_size, max_length)
        
        padding_batch = np.array([ np.pad(inst, (0, padding_size - len(inst)), constant_values=padding_idx) for inst in batch ])
        return padding_batch
    
    @staticmethod
    def build_label(y, classes=None):
        """(static method) build_label
        build labels array using one-hot encoding

        Args:
            y (_ArrayLike): input labels
            classes (_ArrayLike, optional): array contains all classes. if None, it will set to unique label of input. Defaults to None.

        Returns:
            _Array[_Array[int]]: array of one-hot label where shape is (input_size, num_classes)
        """
        if classes is None:
            classes = np.unique(y)
            
        encoder = OneHotEncoder(categories=[classes])
        return encoder.fit_transform(y).toarray()

In [ ]:
prep = LogPreprocessor()
# extract word2vec format file to embedding matrix and words indice
# change this path for select another embedding method
prep.load_word2vec_format("../BGL_word2Vec-fine-tune-embedder.txt")
# insert "<PAD>" at first index of embedding matrix with zeros vector
# if padding token is represented as zeros vector, unknown token must be represented
# as non-zeros vector. (that mean if you set unknow_token for load_word2vec_format you
# have to set unknow_repr too or using non-zero vector for represent padding token).
prep.insert_new_word("<PAD>", np.zeros(prep.embedding_shape.embedding_size), index=0)

In [ ]:
import pandas as pd

# load processed trainset of structured log and template log data
prep_train_struc_log = pd.read_pickle("../data_preprocess/processed_type2/BGL_preprocessed_type2/train_set.pkl")
prep_train_templ_log = pd.read_pickle("../data_preprocess/processed_type2/BGL_preprocessed_type2/template_train_set.pkl")

# encode token to index of embedding matrix
prep_train_struc_log["Token_Indice_encoded"] = prep_train_struc_log.Token.map(prep.indice_encode)
# padding token with "<PAD>"
padd_input = prep.indice_padding(prep_train_struc_log["Token_Indice_encoded"], "<PAD>")
# build label array to one hot format
label = prep.build_label(prep_train_struc_log["Label"].to_numpy().reshape((-1, 1)), classes=[0, 1])
# you also can use this preprocessing pipeline for testset

prep_test_struc_log = pd.read_pickle("../data_preprocess/processed_type2/BGL_preprocessed_type2/test_set.pkl")
prep_test_templ_log = pd.read_pickle("../data_preprocess/processed_type2/BGL_preprocessed_type2/template_test_set.pkl")

prep_test_struc_log["Token_Indice_encoded"] = prep_test_struc_log.Token.map(prep.indice_encode)

padd_test_input = prep.indice_padding(prep_test_struc_log["Token_Indice_encoded"], "<PAD>")

test_label = prep.build_label(prep_test_struc_log["Label"].to_numpy().reshape((-1, 1)), classes=[0, 1])


In [ ]:
class TextLSTMConfig():
    def __init__(self,
                num_classes,
                vocab_size,
                embedding_size, 
                filter_sizes, 
                num_filters,
                sequence_length=None,
                dropout_rate=None,
                l2_reg_lambda=0.0,
                seed=42,
                pretrain_embedding_matrix=None
                ) -> None:
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.sequence_length = sequence_length
        self.dropout_rate = dropout_rate
        self.l2_reg_lambda = l2_reg_lambda
        self.seed = seed
        self.pretrain_embedding_matrix = pretrain_embedding_matrix
       
    def __repr__(self):
        return "TextLSTMConfig:\n" +\
            f"num_classes:      {self.num_classes}\n" +\
            f"vocab_sizes:      {self.vocab_size}\n" +\
            f"embedding_size:   {self.embedding_size}\n" +\
            f"filter_sizes:     {self.filter_sizes}\n" +\
            f"num_filters:      {self.num_filters}\n" +\
            f"sequence_length:  {self.sequence_length}\n" +\
            f"dropout_rate:     {self.dropout_rate}\n" +\
            f"l2_reg_lambda:    {self.l2_reg_lambda}\n" +\
            f"seed:             {self.seed}\n" +\
            f"pretrain_embedding_matrix's shape: {np.shape(self.pretrain_embedding_matrix) if self.pretrain_embedding_matrix is not None else 'None'}"
            
class TextLSTM(object):
    def __init__(self,
        num_classes,
        vocab_size,
        embedding_size, 
        filter_sizes, 
        num_filters,
        sequence_length=None,
        dropout_rate=None,
        l2_reg_lambda=0.0,
        seed=42,
        pretrain_embedding_matrix=None
        ) -> None:
        
        self.config = TextLSTMConfig(
            num_classes,
            vocab_size,
            embedding_size, 
            filter_sizes, 
            num_filters,
            sequence_length,
            dropout_rate,
            l2_reg_lambda,
            seed,
            pretrain_embedding_matrix
        )
        
        if seed is not None: tf.random.set_seed(seed)
        
        input_word_idx = tf.keras.layers.Input(
            shape=(sequence_length,),
            dtype=tf.dtypes.int32,
            name="input-word-idx-layer"
        )
        
        embed_trainable = pretrain_embedding_matrix is None
        if pretrain_embedding_matrix is None:
            embed_initializers = tf.keras.initializers.RandomUniform(minval=-1, maxval=1)
        else:
            pretrain_embedding_matrix = np.array(pretrain_embedding_matrix)
            assert (vocab_size, embedding_size) == pretrain_embedding_matrix.shape, \
                f"shape of embedding_matrix must match to vocab_size and embedding_size (expect {(vocab_size, embedding_size)}, got {pretrain_embedding_matrix.shape})."
            embed_initializers = tf.keras.initializers.Constant(pretrain_embedding_matrix)
            
        with tf.name_scope("embedding"), tf.device("cpu:0"):
            # define embedding layer where in put is array of index of word's vector in
            # embedding matrix. the output's shape is 
            # (batch_size, sequence_length, embedding_size)
            embed = tf.keras.layers.Embedding(
                input_dim=vocab_size,
                output_dim=embedding_size,
                embeddings_initializer=embed_initializers,
                input_length=sequence_length,
                trainable=embed_trainable,
                name="embedding-layer"
            )(input_word_idx)

        with tf.name_scope("convolution"):
            lstm = tf.keras.layers.LSTM()(embed)     
        
        if dropout_rate is not None:
            with tf.name_scope("dropout"):
                # define dropout layer with specified dropout_rate
                dropout = tf.keras.layers.Dropout(rate=dropout_rate)(lstm)
                fc_input = dropout
        else: fc_input = lstm 
        
        with tf.name_scope("fully-connected"):
            # define output layer (fully connected layer) using Softmax as activation
            # and L2 as regularization method. where output is propability to be each 
            # class with output shape is (batch_size, num_classes).
            output = tf.keras.layers.Dense(
                units=num_classes,
                activation="softmax",
                use_bias=True,
                kernel_initializer=tf.keras.initializers.GlorotUniform(),
                bias_initializer=tf.keras.initializers.Constant(0.1),
                kernel_regularizer=tf.keras.regularizers.L2(l2=l2_reg_lambda),
                bias_regularizer=tf.keras.regularizers.L2(l2=l2_reg_lambda),
                name="output-layer"
                )(fc_input)
        
        # define model with all defined sequent layers
        self.model = tf.keras.Model(inputs=input_word_idx, outputs=output)    
        
        
        
        


In [ ]:
textcnn = TextLSTM(
                num_classes=2,
                vocab_size=prep.embedding_shape.vocab_size,
                embedding_size=prep.embedding_shape.embedding_size,
                filter_sizes=[2, 3, 4],
                num_filters=2,
                sequence_length=None,
                dropout_rate=0.5,
                l2_reg_lambda=0.01,
                seed=42,
                pretrain_embedding_matrix=prep.word_vectors)

textcnn.config